## Research for the book developing startup:

The coronavirus took the entire world by surprise, changing everyone's daily routine. City dwellers no longer spent their free time outside, going to cafes and malls; more people were at home, reading books. That attracted the attention of startups that rushed to develop new apps for book lovers. 

In this research we'll work with the database of one of the services competing in this market. It contains data on books, publishers, authors, and customer ratings and reviews of books. This information will be used to generate a value proposition for a new product.

Main questions to answer:
- number of books released after January 1, 2000.
- number of user reviews and the average rating for each book.
- identifying the publisher that has released the greatest number of books with more than 50 pages (excluding brochures and similar publications from your analysis).
- identifying the author with the highest average book rating (books with at least 50 ratings).
- average number of text reviews among users who rated more than 50 books.

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'Sdf4$2;d-d30pp', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
!pip install pymysql

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import pymysql

Let's study tables: 
- books
- ratings
- authours
- reviews
- publishers

Storing result in the data frame, showing first rows.

In [6]:
data_books=f'''SELECT * FROM books'''
books = pd.io.sql.read_sql(data_books, con = engine, index_col = 'book_id')

In [7]:
books_load=pd.read_sql(data_books,engine)
books_load.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [22]:
query = '''SELECT * FROM ratings'''
ratings = pd.io.sql.read_sql(query, con = engine, index_col = 'rating_id')
ratings.head()

,book_id,username,rating
rating_id,,,
1,1,ryanfranco,4
2,1,grantpatricia,2
3,1,brandtandrea,5
4,2,lorichen,3
5,2,mariokeller,2


In [23]:
query = '''SELECT * FROM authors'''
authors = pd.io.sql.read_sql(query, con = engine, index_col = 'author_id')
authors.head()

,author
author_id,
1,A.S. Byatt
2,Aesop/Laura Harris/Laura Gibbs
3,Agatha Christie
4,Alan Brennert
5,Alan Moore/David Lloyd


In [24]:
query = '''SELECT * FROM reviews'''
reviews = pd.io.sql.read_sql(query, con = engine, index_col = 'review_id')
reviews.head()

,book_id,username,text
review_id,,,
1,1,brandtandrea,Mention society tell send professor analysis. ...
2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
4,3,johnsonamanda,Finally month interesting blue could nature cu...
5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [53]:
query = '''SELECT * FROM publishers'''
publishers = pd.io.sql.read_sql(query, con = engine, index_col = 'publisher_id')
publishers.head()

,publisher
publisher_id,
1,Ace
2,Ace Book
3,Ace Books
4,Ace Hardcover
5,Addison Wesley Publishing Company


### Number of books released after January 1, 2000

In [20]:
books_released = f''' SELECT COUNT(book_id) AS book_amount
FROM 
    books
WHERE 
    CAST(books.publication_date AS date) >= '2000-01-01' '''
books_data=pd.read_sql(books_released,engine)
books_data

,book_amount
0,821


### Number of user reviews and the average rating for each book

In [54]:
reviews_rating = f''' SELECT 
    books.title,
    COUNT(DISTINCT review_id) AS reviews_amount,
    AVG(rating) AS average_rating
FROM 
    books
LEFT JOIN 
    reviews 
ON 
    books.book_id = reviews.book_id
RIGHT JOIN 
    ratings
ON
    books.book_id = ratings.book_id
GROUP BY 
    books.book_id
ORDER BY 
    reviews_amount DESC 
LIMIT
    10
    
    '''
reviews_rating_data=pd.read_sql(reviews_rating,engine)
reviews_rating_data

,title,reviews_amount,average_rating
0,Twilight (Twilight #1),7,3.662500
1,The Da Vinci Code (Robert Langdon #2),6,3.830508
2,Eat Pray Love,6,3.395833
3,The Alchemist,6,3.789474
4,The Catcher in the Rye,6,3.825581
5,The Curious Incident of the Dog in the Night-Time,6,4.081081
6,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
7,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
8,Outlander (Outlander #1),6,4.125000
9,The Book Thief,6,4.264151


We can see that top 3 leaders for the reviews amount are following books:
- Twilight: 7 reviews with average rating of 3.66;
- The Da Vinci Code: 6 reviews with average rating of 3.83;
- Eat Pray Love: 6 reviews with average rating of 3.39.


### Publisher who released the greatest number of books with more than 50 pages

In [59]:
publisher_max50p_books = f''' SELECT 
    publishers.publisher,
    COUNT(DISTINCT book_id) AS books_count
FROM 
    books
LEFT JOIN 
    publishers 
ON 
    books.publisher_id = publishers.publisher_id
WHERE
    books.num_pages > '50'
GROUP BY 
    publishers.publisher_id
ORDER BY 
    books_count DESC 
LIMIT
    3
    '''
publisher_max50p_books_data=pd.read_sql(publisher_max50p_books,engine)
publisher_max50p_books_data

,publisher,books_count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25


While checking publishers, who released the greatest number of books with more than 50 pages (excluding brochures and similar publications from our analysis), we saw following top 3:
- Penguin Books	: 42 books
- Vintage : 31 books
- Grand Central Publishing : 25 books

### Author with the highest average book rating: books with at least 50 ratings


In [78]:
author_rating = f''' SELECT 
    authors.author,
    AVG(ratings.rating) AS average_rating,
    COUNT(DISTINCT ratings.rating_id) as rating_count
FROM 
    books
LEFT JOIN 
    authors 
ON 
    books.author_id = authors.author_id
LEFT JOIN 
    ratings
ON
    books.book_id = ratings.book_id
GROUP BY 
    books.book_id,
    books.author_id,
    authors.author
HAVING
    COUNT(DISTINCT ratings.rating_id) > '50'
ORDER BY 
    average_rating DESC 
LIMIT
    1
    
    '''
author_rating_data=pd.read_sql(author_rating,engine)
author_rating_data

,author,average_rating,rating_count
0,J.K. Rowling/Mary GrandPré,4.414634,82


While checking the author with the highest average book rating (books with at least 50 ratings) we found the author: J.K. Rowling/Mary GrandPré	.


### Average number of text reviews among users who rated more than 50 books

In [120]:
reviews_mo50books = f''' SELECT 
    ROUND(AVG(subquery.text_reviews_count),2) as avg_text_reviews_count
FROM
    (SELECT 
        COUNT(DISTINCT reviews.review_id) as text_reviews_count
    FROM 
        reviews
    INNER JOIN 
        ratings 
    ON 
        reviews.username = ratings.username
    GROUP BY 
        ratings.username
    HAVING 
        COUNT(DISTINCT ratings.rating_id) > '50') as subquery
    '''
reviews_mo50books_data=pd.read_sql(reviews_mo50books,engine)
reviews_mo50books_data

,avg_text_reviews_count
0,24.33


While checking the average number of text reviews among users who rated more than 50 books, we found that it is 24.33.

## Conclusion

In this research for the book developing startup we worked with 5 datasets from the database of one of the services competing in this market. It contained data on books, publishers, authors, and customer ratings and reviews of books. This information was used to generate a value proposition for a new product and answer following questions:

- number of books released after January 1, 2000: **821**
- number of user reviews and the average rating for each book - top 3:
    - **Twilight**: 7 reviews with average rating of 3.66
    - **The Da Vinci Code**: 6 reviews with average rating of 3.83
    - **Eat Pray Love**: 6 reviews with average rating of 3.39
- publisher that has released the greatest number of books with more than 50 pages: **Penguin Books**
- author with the highest average book rating (books with at least 50 ratings): **J.K. Rowling/Mary GrandPré**
- average number of text reviews among users who rated more than 50 books: **24.33**